In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt, cm, colors
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D

sns.set(style="darkgrid")
sns.set(style="darkgrid")
from matplotlib import pyplot as plt, cm, colors
from tqdm.notebook import tqdm

import glob
import os.path as ospath
import os
from sys import executable
from subprocess import check_output
from PyQt5.QtWidgets import QFileDialog, QApplication
from IPython.display import HTML

from scipy import optimize
from scipy.spatial import distance
from scipy import linalg
from scipy import signal
from sklearn.cluster import MeanShift, estimate_bandwidth

from picasso.picasso import io
from picasso.picasso.postprocess import link, compute_dark_times
from picasso.picasso.render import render
from picasso.picasso.gui.render import estimate_kinetic_rate, fit_cum_exp



In [2]:
# define visualization options 
%matplotlib inline
%gui qt

# define colors:
blue = "#4C72B0"
orange = "#DD8452"
red = "#C44E52"
gray = "#90A8CE"


def OpenFileDialog():
    file = check_output([executable, __file__])
    return file.strip()


def gui_fname(dir=None):
    """
    Select a file via a dialog and return the file name.
    """
    if dir is None: 
        dir ="./"

    app = QApplication([dir])
    fname = QFileDialog.getExistingDirectory(None, "Select a folder...", 
            dir)
    if isinstance(fname, tuple):
        return fname[0]
    else: 
        return str(fname)


def load_files(dirname):
    
    os.chdir(dirname)
    files = glob.glob("*.hdf5")
    
    if files:
        print("{} HDF5 files found.".format(len(files)))
    else:
        print("No HDF5 files found at: {}".format(dirname))
            
    return files


def load_data(path):

    data, info = io.load_locs(path)
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")
  

    # convert px to nm
    data.x *= pixelsize
    data.y *= pixelsize
    data.x_pick_rot *= pixelsize
    data.y_pick_rot *= pixelsize
    
    return data, info, pixelsize


def double_gaus(x,a,x0,sigma, b, x1, sigma1):
    return a*np.exp(-(x-x0)**2/(2*sigma**2)) + b*np.exp(-(x-x1)**2/(2*sigma1**2))

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2)) 


def find_peaks(data, binning=100, axes="y"):
    
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
    elif axes == "xyz":
        column = 2
    
    # find peaks
    bandwidth = estimate_bandwidth(data[column].reshape(-1, 1), quantile=0.2, n_samples=binning)
    #print("estimated bandwidth: "+str(bandwidth))
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(data[column].reshape(-1, 1))
    labels = ms.labels_
    peaks = np.sort(ms.cluster_centers_[0:2],axis=None) # assuming that there are two large peaks from the two rings
    peak1 = float(peaks[0])
    peak2 = float(peaks[1])
    estimated_peaks = {0:peak1, 1:peak2}

    # histogram
    n, bins = np.histogram(data[column], bins=binning)
    #bins = bins[1:]
    centers = (bins[:-1] + bins[1:]) / 2
    hist_data = [n, centers]

    # fit peaks
    p0 = [peak1/2, peak1, 40, peak2/2, peak2, 40]
    try:
        p_fit, p_cov = optimize.curve_fit(double_gaus, centers, n, p0=p0)
    except:
        p_fit = [0,0,0,1000,1000,1000]
        
    
    # check order of fitted peaks (peak 1 < peak 2)
    if p_fit[1] > p_fit[4]:
        p_temp = p_fit.copy()
        p_fit[0:3]=p_temp[3:6]
        p_fit[3:6]=p_temp[0:3]
       
        
    # p_fit: amplitued_1, center_1, width_1, amplitude_2, center_2, width_2
        
    return estimated_peaks, p_fit, hist_data

def find_peak(data, binning=100, axes="y"):
    
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
    elif axes == "xyz":
        column = 2
    
    # find peak
    bandwidth = estimate_bandwidth(data[column].reshape(-1, 1), quantile=0.2, n_samples=binning)
    #print("estimated bandwidth: "+str(bandwidth))
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(data[column].reshape(-1, 1))
    labels = ms.labels_
   # print(ms.cluster_centers_[0:2])
    peaks = np.sort(ms.cluster_centers_[0:2], axis=None)
    peak1 = float(ms.cluster_centers_[0])
    estimated_peak = {0:peak1}

    # histogram
    n, bins = np.histogram(data[column], bins=binning)
    #bins = bins[1:]
    centers = (bins[:-1] + bins[1:]) / 2
    hist_data = [n, centers]

    # fit peaks
    p0 = [peak1/2, peak1, 40]
    try:
        p_fit, p_cov = optimize.curve_fit(gaus, centers, n, p0=p0)
    except:
        p_fit = [0,0,0]
        
    return estimated_peak, p_fit, hist_data


def plot_peak_dist(data, hist_data, peaks, p_fit, binning=100,cutoff=1, axes="y", ax=None):
    
    if ax is None:
        ax = plt.gca()
        
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
    elif axes == "xyz":
        column == 1
        
    peak1 = peaks[0]
    peak2 = peaks[1]
    
    n = hist_data[0]
    bins = hist_data[1]
    
    #fig, ax = plt.subplots(figsize=(9, 5))
    binwidth = (max(bins)-min(bins)) / binning
    ax.bar(bins, n, width=binwidth, color=gray)
    xlin = np.linspace(0, data[column].max(), 1000)
    ax.plot(xlin, gaus(xlin,*p_fit[0:3]), c=red, linewidth=2)
    ax.plot(xlin, gaus(xlin,*p_fit[3:6]), c=red, linewidth=2)
    ax.axvline(p_fit[1]-(p_fit[2]*cutoff),c=blue, linewidth=2, linestyle="--")
    ax.axvline(p_fit[1]+(p_fit[2]*cutoff),c=blue, linewidth=2, linestyle="--")
    ax.axvline(p_fit[4]-(p_fit[5]*cutoff),c=orange, linewidth=2, linestyle="--")
    ax.axvline(p_fit[4]+(p_fit[5]*cutoff),c=orange, linewidth=2, linestyle="--")
    ax.set_title("Line profile",loc="left",fontsize=14)
    ax.set_xlabel("y (nm)")
    ax.set_ylabel("Counts")  
    ax.text(0.15,
            0.7,
            ("Estimated Peaks:\n"
            "Peak 1 at {:.1f} nm\n"
            "Peak 2 at {:.1f} nm\n"
            "Fitted Peaks:\n"
            "Peak 1 at {:.1f} nm\n"
            "Peak 2 at {:.1f} nm").format(peak1,peak2,p_fit[1],p_fit[4]),
            horizontalalignment="center",
            verticalalignment="center",
            transform = ax.transAxes,
            fontsize=12)

    return ax


def isolate_ring(data, c, w, axes="y", cutoff=1.0):
    
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
        
    ring_data = data[np.where((data[column]>(c-(cutoff*w))) & (data[column] <(c+(cutoff*w))))]
    return ring_data


def rodrigues_rot(P, n0, n1):
    # adapted from https://meshlogic.github.io/posts/jupyter/curve-fitting/fitting-a-circle-to-cluster-of-3d-points/
    # If P is only 1d array (coords of single point), fix it to be matrix
    if P.ndim == 1:
        P = P[np.newaxis,:]
    
    # Get vector of rotation k and angle theta
    n0 = n0/linalg.norm(n0)
    n1 = n1/linalg.norm(n1)
    k = np.cross(n0,n1)
    k = k/linalg.norm(k)
    theta =np.arccos(np.dot(n0,n1))
    
    # Compute rotated points
    P_rot = np.zeros((len(P),3))
    for i in range(len(P)):
        P_rot[i] = P[i]*np.cos(theta) + np.cross(k,P[i])*np.sin(theta) + k*np.dot(k,P[i])*(1-np.cos(theta))

    return P_rot


def rotate_ring(XYZ): 
    # Fitting plane by SVD for the mean-centered data
    # Eq. of plane is <p,n> + d = 0, where p is a point on plane and n is normal vector
       
    # Normal vector of fitting plane is given by 3rd column in V
    # Note linalg.svd returns V^T, so we need to select 3rd row from V^T
    ring_mean = XYZ.mean(axis=0)
    ring_centered = XYZ - ring_mean
    U,s,V = linalg.svd(ring_centered)
    normal = V[2,:]
    d = -np.dot(ring_mean, normal) 
        
    n0 = normal # new z axes
    n1 = [0,0,1] # old z axes

    ring_rot = rodrigues_rot(ring_centered, n0, n1)
    
    return ring_rot, normal


def plot_3d_ring(ring_rot, dist, ax=None):
    
    ax.scatter(ring_rot[0][:,0], ring_rot[0][:,1], ring_rot[0][:,2]-dist/2,c=blue,label="Spore", alpha=0.5)
    ax.scatter(ring_rot[1][:,0], ring_rot[1][:,1], ring_rot[1][:,2]+dist/2,c=orange,label="Mother", alpha=0.5)
    ax.set_xlabel("x (nm)")
    ax.set_ylabel("y (nm)")
    ax.set_zlabel("z (nm)")
    ax.legend(loc="best",labelspacing=0.1)
    #ax.view_init(elev=2., azim=10.)
    ax.view_init(elev=30., azim=10.)
    set_axes_equal_3d(ax)

    return ax
    
    
def set_axes_equal_3d(ax):
    limits = np.array([ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()])
    spans = abs(limits[:,0] - limits[:,1])
    centers = np.mean(limits, axis=1)
    radius = 0.5 * max(spans)
    ax.set_xlim3d([centers[0]-radius, centers[0]+radius])
    ax.set_ylim3d([centers[1]-radius, centers[1]+radius])
    ax.set_zlim3d([centers[2]-radius, centers[2]+radius])
    

def angle_between(u, v, n=None):
    if n is None:
        return np.arctan2(np.linalg.norm(np.cross(u,v)), np.dot(u,v))*180/np.pi
    else:
        return np.arctan2(np.dot(n,np.cross(u,v)), np.dot(u,v))*180/np.pi


def calc_R(x, y, xc, yc):
    # adapted from https://gist.github.com/lorenzoriano/6799568
    """ calculate the distance of each 2D points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)


def f(c, x, y):
    """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
    Ri = calc_R(x, y, *c)
    return Ri - np.median(Ri)


def leastsq_circle(x,y):
    
    x_m = np.median(x)
    y_m = np.median(y)
    center_estimate = x_m, y_m
    center, ier = optimize.leastsq(f, center_estimate, args=(x,y))
    xc, yc = center
    Ri       = calc_R(x, y, *center)
    R        = np.median(Ri)
    residu   = np.sum((Ri - R)**2)
    return xc, yc, R, residu


def plot_data_circle(x, y, xc, yc, R, id, center=True, ax=None):
    
    if id == 0:
        label = "spore"
        color = blue
    else:
        label = "mother"
        color = orange
    
    if ax is None:
        ax = plt.gca()
    
    if center:
        x -= xc
        y -= yc
        xc = 0
        yc = 0 
        
    #f, ax = plt.subplots(figsize=(5,5))  #figsize=(7, 5.4), dpi=72,
    theta_fit = np.linspace(-np.pi,np.pi, 180)
    x_fit = xc + R*np.cos(theta_fit)
    y_fit = yc + R*np.sin(theta_fit)
    
    # plot fit
    ax.plot(x_fit, y_fit, label="Fitted circle", lw=2, c=red)
    ax.plot([xc], [yc], mec="y", mew=1,  c=red)
    
    # plot data
    ax.scatter(x, y, alpha=0.4,  label="Projected locs", marker=".", c=color)
    
    ax.set_xlabel("x rotated (nm)")
    ax.set_ylabel("y rotated (nm)")
    #ax.axis("equal")
    ax.set_xlim([-800,800])
    ax.set_aspect("equal",adjustable="datalim")
    
    ax.legend(loc="best", labelspacing=0.1)
    ax.set_title("Least squares circle {}\n"
                 "Fit radius: {:.1f} nm".format(label,R),loc="left",fontsize=14)
    
    return 


def check_consistency(ring_par):
    
    ev = 0
    
    amp_1, c_1, w_1 = ring_par[0]
    amp_2, c_2, w_2 = ring_par[1]
    
    if amp_1/amp_2 >= 0.3 and amp_2/amp_1 >= 0.3:
        ev +=1 
        
    if abs(c_1-c_2) > 30 and abs(c_1-c_2) < 300:
        ev +=1
        
    if w_1/w_2 >= 0.3 and w_2/w_1 >=0.3:
        ev +=1
        
    return (ev==3)

def plot_cum_exp(pooled_locs, fit_result_len, fit_result_dark, id, ax=None):
    
    if id == 0:
        color = blue
        label = 'spore'
    if id == 1:
        color = orange
        label = 'mother'

    if ax is None:
        ax = plt.gca()
    
    data = pooled_locs.dark
    data.sort()
    y = np.arange(1, len(data) + 1)
       
    a = fit_result_dark.best_values["a"]
    t = fit_result_dark.best_values["t"]
    c = fit_result_dark.best_values["c"]

    ax.set_title(
        "Dark time (cumulative) {}\n"
        r"$Fit: {:.2f}\cdot(1-exp(x/{:.2f}))+{:.2f}$".format(label, a, t, c),loc="left",fontsize=14)
    data = pooled_locs.dark
    data.sort()
    y = np.arange(1, len(data) + 1)

    ax.semilogx(data, y, c=color, label="Data")
    ax.semilogx(data, fit_result_dark.best_fit, c=red, label="Fit")
    ax.legend(loc="best")
    ax.set_xlabel("Duration (frames)")
    ax.set_ylabel("Frequency")
    
    return ax


def save_ring_locs(locs, info, path, file_id, pick, id, link=False):
    
    if link:
        ending = "_link.hdf5"
    else:
        ending = ".hdf5"
       
    locs.x /= 130
    locs.y /= 130
    
    locs_name = "file_{}_pick_{}_ring_{}{}".format(file_id, pick, id, ending)
    locs_path = os.path.join(path,"ring_locs")
    locs_path_name = os.path.join(locs_path, locs_name)
    
    if not os.path.isdir(locs_path):
        os.makedirs(locs_path)
    
    io.save_locs(locs_path_name, locs, info)

    
def export_pick_img(locs, path, file_id, pick, id, link=False):
    
    if link:
        ending = "_link.png"
    else:
        ending = ".png"
    
    pixelsize = 130

    export_locs = locs.copy()
    
    export_locs.x /= pixelsize
    export_locs.y /= pixelsize
    
    x_min = np.min(export_locs.x)    
    x_max = np.max(export_locs.x)
    y_min = np.min(export_locs.y)
    y_max = np.max(export_locs.y)

    viewport =  (y_min, x_min), (y_max, x_max)
    oversampling = 50
    len_x, image = render(export_locs, viewport = viewport, oversampling=oversampling, blur_method="smooth")
    
    img_name = "file_{}_pick_{}_ring_{}{}".format(file_id, pick, id, ending)
    img_path = os.path.join(path,"ring_images")
    img_path_name = os.path.join(img_path,img_name)
    
    if not os.path.isdir(img_path):
        os.makedirs(img_path)
    
    plt.imsave(img_path_name, image, cmap="hot")

## Load data

In [3]:
#path = gui_fname()
path = r'X:\users\kcramer\sporPAINT\DivIVA_FtsZ\spor'

div_path = os.path.join(path, "div")
fts_path = os.path.join(path, "fts")
div_filenames = load_files(div_path)
fts_filenames = load_files(fts_path)

plotting = True
max_dist = 130 #nanometer
max_dark_time = 15 #frames
binning = 50 # binning for peak histogram


34 HDF5 files found.
34 HDF5 files found.


## Main loop

In [4]:
# initialize containers
ring_data, fts_ring_data = [], []
ring_locs,fts_ring_locs = {}, {}
ring_locs_linked, fts_ring_locs_linked = {}, {}
ring_kinetics, fts_ring_kinetics = {}, {}
ring_kinetics_fit, fts_ring_kinetics_fit = {}, {}
ring_radii, fts_ring_radii = {}, {}
ring_rot, fts_ring_rot = {}, {}
ring_angles,fts_ring_angles = {}, {}
center, fts_center = {}, {}

rings_excluded, fts_rings_excluded = [], []
ring_n_events, fts_ring_n_events = {}, {}
circle_plots, fts_circle_plots = {}, {}
file_id = 0

analysis_folder = os.path.join(path, "analysis")
analysis_folder_excluded = os.path.join(analysis_folder, "excluded")

# image export settings
img_format = ".png"
dpi = 100

# cutoff is mutliplied to the sigma of the peak fit for the seperation of the two rings
# smaller cutoff means smaller ring sections
cutoff = 1.25

#prepare analysis folder
if not os.path.isdir(analysis_folder):
    os.makedirs(analysis_folder)
    os.makedirs(analysis_folder_excluded)

#iterate over locs in directory:
for filename in tqdm(div_filenames, desc="Processing files"):
    
    print("Current file: {}".format(filename))
    
    file_id +=1
    
    #load DIV4A locs and convert distances from px to nm (Attention!)
    locs, info, pixelsize = load_data(os.path.join(div_path, filename))
    
    #look for corresponding FtsZ file
    fts_filename = filename.replace('DivIVA','ftsz')
    
    try:
        fts_locs, fts_info, fts_pixelsize = load_data(os.path.join(fts_path, fts_filename))
    except:
        print('ERROR: Could not find FtsZ file for {}\n Check file names again.'.format(filename))
    
    # iterate over picks in a file
    for pick in tqdm(np.unique(locs.group), desc="Processing picks"):
     
        # select locs from pick
        pick_locs = locs[locs.group == pick]
        fts_pick_locs = fts_locs[fts_locs.group == pick]
        
        ################
        # (1) estimate the postition of the two rings using a histogram along the pick direction
        #  a ring should yield a gaussian peak (2D projection in XY)
        ################
        
        estimated_peaks, r_par, hist_data = find_peaks(pick_locs, binning=binning, axes="y")
        try:
            fts_estimated_peak, fts_r_par, fts_hist_data = find_peak(fts_pick_locs, binning=binning, axes="y")
        except:
            print('ERROR: Could not find peak for FtsZ ring. Pick will be ignored')
            continue
            
        # estimated peaks form MeanShift analysis to initialize guassian fits
        # r_par yields an array containing: [amplitued_1, center_1, width_1, amplitude_2, center_2, width_2]
        
        # seperate ring parameter to the corresponding ring
        ring_parameter = []
        for i in range(0, 4, 3):
            ring_parameter.append(r_par[i: i+3])
        print(ring_parameter)
        fts_ring_parameter = [] # to work as the double band script
        fts_ring_parameter.append(fts_r_par)
            
        if check_consistency(ring_parameter):
        # check three criteria:
        # i   distance between the peaks (30 nm < distance < 300 nm)
        # ii  width of the gaussian fits of the peaks (ratio needs to be larger then 0.2 in both cases)
        # iii "signal strength" of both rings should be similar (amplitude of the gaussian peak should be larger then 0.3)
            
            # set up plot with gridspec
            fig = plt.figure(figsize=(14, 16), constrained_layout=True)
            gs = fig.add_gridspec(3, 2)
            fig.suptitle(("Pick {} - Ring analysis\n"
                          "File: {}").format(pick,filename),
                          fontsize=16,
                          ha="center")
            
            ax1 = fig.add_subplot(gs[0, 0])
           
            plot_peak_dist(pick_locs,
                           hist_data,
                           estimated_peaks,
                           r_par,
                           binning=binning,
                           cutoff=cutoff,
                           axes="y",
                           ax=ax1)
            
            peak_dist = abs(ring_parameter[0][1]-ring_parameter[1][1])
            fts_peak_dist = fts_ring_parameter[0][1]-ring_parameter[0][1]
        
            ################
            # (2) isolate the locs from every ring
            # fitted center +- peak-to-peak distance/2
            ################
            
            #DIV4A
            for id, ring in enumerate(ring_parameter):
                
                # isolate the locs from every ring
                r_locs = isolate_ring(pick_locs, c=ring[1], w=ring[2], cutoff=cutoff, axes="y")
                ring_locs[id] = r_locs
                
                # link the isolated locs
                r_locs_linked = link(r_locs, info, max_dist, max_dark_time)
                ring_locs_linked[id] = r_locs_linked
                
                # extract number of locs
                ring_n_events[id] = len(r_locs_linked)
             
            #FtsZ
            for fts_id, fts_ring in enumerate(fts_ring_parameter):
    
                # isolate the locs from every ring
                fts_r_locs = isolate_ring(fts_pick_locs, c=fts_ring[1], w=fts_ring[2], cutoff=cutoff, axes="y")
                fts_ring_locs[fts_id] = fts_r_locs
                
                # link the isolated locs
                fts_r_locs_linked = link(fts_r_locs, fts_info, max_dist, max_dark_time)
                fts_ring_locs_linked[fts_id] = fts_r_locs_linked
                
                # extract number of locs
                fts_ring_n_events[fts_id] = len(fts_r_locs_linked)
            
              
            ################
            # (3b) calculate ring radius by rotation into plane of the ring and circle fitting in 2D projection
            ################
            
            ax3 = fig.add_subplot(gs[1, 0])
            ax5 = fig.add_subplot(gs[2, 0])
            normal = {}
            fts_normal = {}
            
            for id, rings in ring_locs.items():
                
                # prepare XYZ coordinates
                XYZ = np.array([rings.x, rings.y, rings.z]).transpose()

                # rotate coordinates into the plane of the ring. ring should be now in X-Y
                ring_rot[id], normal[id] = rotate_ring(XYZ)
                
                # fit 2D ring into the XY ring data
                x_center, y_center, Radius, residu = leastsq_circle(ring_rot[id][:,0], ring_rot[id][:,1])
                
                c = rodrigues_rot(np.array([x_center, y_center, 0]), [0,0,1], normal[id]) + XYZ.mean(axis=0)
                center[id] = c.flatten()
                
                # extract radii
                ring_radii[id] = Radius
                
                # plot
                if id == 0:
                    ax_circ = ax3
                if id == 1:
                    ax_circ = ax5
                
                plot_data_circle(ring_rot[id][:,0],
                                 ring_rot[id][:,1],
                                 x_center,
                                 y_center,
                                 Radius,
                                 id,
                                 center=True,
                                 ax=ax_circ)
                
            for fts_id, fts_rings in fts_ring_locs.items():
                
                # prepare XYZ coordinates
                fts_XYZ = np.array([fts_rings.x, fts_rings.y, fts_rings.z]).transpose()

                # rotate coordinates into the plane of the ring. ring should be now in X-Y
                fts_ring_rot[fts_id], fts_normal[fts_id] = rotate_ring(fts_XYZ)
                
                # fit 2D ring into the XY ring data
                fts_x_center, fts_y_center, fts_Radius, fts_residu = leastsq_circle(fts_ring_rot[fts_id][:,0],
                                                                                    fts_ring_rot[fts_id][:,1])
                
                fts_c = rodrigues_rot(np.array([fts_x_center, fts_y_center,0]), 
                                      [0,0,1], 
                                      fts_normal[fts_id]) + fts_XYZ.mean(axis=0)
                
                fts_center[fts_id] = fts_c.flatten()
                
                # extract radii
                fts_ring_radii[fts_id] = fts_Radius
                
            
            # calculate the center-to-center distance
            center_dist = distance.euclidean(center[0], center[1])
            fts_center_dist = distance.euclidean(center[0], fts_center[0])
                
            # calculate the angle between to two fitted ring planes
            ring_angle = abs(angle_between(normal[0],normal[1]))
            
            # if normals point into opposite direction
            if ring_angle > 90:
                ring_angle -=180
                ring_angle = abs(ring_angle)
               
            # calculate the angle between coverslip and spore or mother ring
            coverslip_s_angle = angle_between(normal[0],[0,0,1])
            coverslip_m_angle = angle_between(normal[1],[0,0,1])
            
            # plot 3D ring
            ax2 = fig.add_subplot(gs[0, 1], projection="3d")
            plot_3d_ring(ring_rot,center_dist, ax=ax2)
            
            ################
            # (4) qPAINT analysis
            ################
            
            ax4 = fig.add_subplot(gs[1, 1])
            ax6 = fig.add_subplot(gs[2, 1])
            
            for id, rings_linked in ring_locs_linked.items():

                    # compute kinetics
                    kinetics = compute_dark_times(rings_linked)

                    # compute mean bright and dark times
                    ring_kinetics[id] = {"bright":np.nanmean(kinetics.len),
                                         "dark":np.nanmean(kinetics.dark)}
                    
                    fit_result_len = fit_cum_exp(kinetics.len)
                    fit_result_dark = fit_cum_exp(kinetics.dark)
                
                    if id == 0:
                        ax_fit=ax4
                    if id == 1:
                        ax_fit=ax6
                    
                    plot_cum_exp(kinetics, fit_result_len, fit_result_dark, id, ax=ax_fit)
                    
                    ring_kinetics_fit[id] = {"bright":np.nanmean(fit_result_len.best_values["t"]),
                                             "dark":np.nanmean(fit_result_dark.best_values["t"])}
                    
            for fts_id, fts_rings_linked in fts_ring_locs_linked.items():

                    # compute kinetics
                    fts_kinetics = compute_dark_times(fts_rings_linked)

                    # compute mean bright and dark times
                    fts_ring_kinetics[fts_id] = {"bright":np.nanmean(fts_kinetics.len),
                                         "dark":np.nanmean(fts_kinetics.dark)}
                    
                    fts_fit_result_len = fit_cum_exp(fts_kinetics.len)
                    fts_fit_result_dark = fit_cum_exp(fts_kinetics.dark)
                    
                    fts_ring_kinetics_fit[fts_id] = {"bright":np.nanmean(fts_fit_result_len.best_values["t"]),
                                             "dark":np.nanmean(fts_fit_result_dark.best_values["t"])}
                       
            ################
            # (5) append data into large array
            ################
            
            ring_data.append([file_id, #running file index
                              filename, #DIV4A filename,
                              fts_filename, #FtsZ filename
                              pick, #pick number
                              ring_radii[0], #fitted radius of spore (nm)
                              ring_radii[1], #fitted radius of mother (nm)
                              fts_ring_radii[0], #fitted radius of ftsZ (nm)
                              peak_dist, #peak-to-peak distance (nm)
                              fts_peak_dist, #spore_peak-to-ftsZ_peak (nm)
                              center_dist, #center-to-center distance (nm)
                              fts_center_dist, #spore_center-to-ftsZ_center (nm)
                              ring_angle, #angle between the two ring planes (°)
                              coverslip_s_angle, #anle between spore ring and coverslip (°)
                              coverslip_m_angle, #anle between mother ring and coverslip (°)
                              ring_n_events[0], #number of locs in spore ring
                              ring_n_events[1], #number of  locs in mother ring
                              fts_ring_n_events[0], #number of locs in ftsZ ring
                              ring_kinetics[0]["bright"], #mean bright time of spore ring (frames)
                              ring_kinetics[0]["dark"], #mean dark time of spore ring (frames)
                              ring_kinetics[1]["bright"], #mean bright time of mother ring (frames)
                              ring_kinetics[1]["dark"], #mean dark time of mother ring (frames)
                              fts_ring_kinetics[0]["bright"], #mean bright time of ftsZ ring (frames)
                              fts_ring_kinetics[0]["dark"], #mean dark time of ftsZ ring (frames)
                              ring_kinetics_fit[0]["bright"], #cumulative mean bright times of spore (frames)
                              ring_kinetics_fit[0]["dark"], #cumulative mean dark times of spore (frames)
                              ring_kinetics_fit[1]["bright"], #cumulative mean bright times of mother (frames)
                              ring_kinetics_fit[1]["dark"], #cumulative mean dark times of mother (frames)
                              fts_ring_kinetics_fit[0]["bright"], #cumulative mean bright times of ftsZ (frames)
                              fts_ring_kinetics_fit[0]["dark"], #cumulative mean dark times of ftsZ (frames)
                             ])
            
            ################
            # (6) save plots
            ################
            
            img_fname = "file_{}_pick_{}".format(file_id, pick)
            img_name = os.path.join(analysis_folder, img_fname)
            plt.savefig(img_name+img_format, dpi=dpi, format="png")
            plt.close(fig)
            
            
            ################
            # (7) export ring locs
            ################
            
            for id, rings_linked in ring_locs_linked.items():
                save_ring_locs(rings_linked, info, analysis_folder, file_id, pick, id, link=True)
            
            for id, rings in ring_locs.items():
                save_ring_locs(rings, info, analysis_folder, file_id, pick, id, link=False)
            
        else:
            fig, ax = plt.subplots(1, 1, figsize=(7, 5), constrained_layout=True)
            fig.suptitle(("Pick {} - Ring analysis\n"
                          "File: {}").format(pick,filename), fontsize=14, ha="center")
            
            plot_peak_dist(pick_locs, hist_data, estimated_peaks, r_par, binning=binning, axes="y", ax=ax)
            img_fname = "file_{}_pick_{}".format(file_id, pick)
            img_name = os.path.join(analysis_folder_excluded, img_fname)
            plt.savefig(img_name+img_format, format="png")
            plt.close(fig)
            rings_excluded.append(pick)
        
print("Calculation finished.")
print("Total: {} rings analysed, {} excluded.".format(len(ring_data), len(rings_excluded)))



# Save analysis data
df_ring_data = pd.DataFrame(ring_data, columns=["file_index",
                                                "div filename",
                                                "fts filename",
                                                "group",
                                                "radius_spore",
                                                "radius_mother",
                                                "radius_ftsZ",
                                                "p2p distance",
                                                "ftsZ distance-p2p spore",
                                                "c2c distance",
                                                "ftsZ distance-c2c spore",
                                                "angle_between_rings",
                                                "angle_between_s_ring_and_coverslip",
                                                "angle_between_m_ring_and_coverslip",
                                                "n_events_spore",
                                                "n_events_mother",
                                                "n_events_ftsZ",
                                                "mean_bright_spore",
                                                "mean_dark_spore",
                                                "mean_bright_mother",
                                                "mean_dark_mother",
                                                "mean_bright_ftsZ",
                                                "mean_dark_ftsZ",
                                                "fit_bright_spore",
                                                "fit_dark_spore",
                                                "fit_bright_mother",
                                                "fit_dark_mother",
                                                "fit_bright_ftsZ",
                                                "fit_dark_ftsZ"
                                               ])
df_ring_data.to_csv("exchange_data.csv")
print("Data saved to CSV file in locs folder.")

Processing files:   0%|          | 0/34 [00:00<?, ?it/s]

Current file: 200717_exch_kcb300_fov1_dp_DivIVA_1_drift_aligned_picked.hdf5


Processing picks:   0%|          | 0/4 [00:00<?, ?it/s]

[array([16.76459616, 99.64642665, 42.24654445]), array([ 90.55895446, 251.28649515,  60.7716169 ])]
[array([ 65.12153427, 145.0869123 ,  17.74042462]), array([ 73.16743622, 250.14547834,  38.5088695 ])]
[array([ 71.75146411, 163.17093649,  22.92096593]), array([ 51.06819938, 244.54088679,  14.80385912])]


Z:\users\reinhardt\z.software\PythonEnvironment\envs\spor_paint\lib\site-packages\scipy\optimize\minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


...

[array([ 84.16148411, 139.20386786,  43.35293893]), array([ 66.42157634, 260.60940679,  46.83030892])]
[array([ 87.63374992, 172.32010172,  34.68232611]), array([ 74.95923542, 293.40879306,  36.64425333])]
[array([346.13649994, 172.46110323,  14.11029445]), array([201.706432  , 230.00962096,  80.9816994 ])]
[array([133.2992661 , 164.64939874,  37.0638056 ]), array([186.81888778, 268.58005835,  28.50380098])]
[array([211.73940799, 187.44100035,  19.7855038 ]), array([ 86.70524701, 248.91686023,  76.29390457])]
Current file: spor_DivIVA_DivIVAandFtsZ_230516_apollo_kcb300_fov1_400pM-r2_DivIVA_DP_1_drift_aligned3_picked.hdf5
ERROR: Could not find FtsZ file for spor_DivIVA_DivIVAandFtsZ_230516_apollo_kcb300_fov1_400pM-r2_DivIVA_DP_1_drift_aligned3_picked.hdf5
 Check file names again.


Processing picks:   0%|          | 0/14 [00:00<?, ?it/s]

[array([379.11605805, 270.91160454, 116.25526811]), array([-92.81559407, 428.92971168,  18.42015815])]
[array([161.68596637, 219.16411148,  77.1990888 ]), array([268.95729686, 308.72510594,  12.97426309])]
[array([175.078115  , 144.90586459,  31.66184199]), array([153.59827505, 242.90724257,  22.64857341])]
[array([206.93118437, 148.28879991,  28.51082867]), array([146.26812275, 248.92486863,  29.46769354])]
[array([110.33850829, 164.13992602,  19.36270219]), array([138.97259318, 284.02225527,  71.45799096])]
ERROR: Could not find peak for FtsZ ring. Pick will be ignored
ERROR: Could not find peak for FtsZ ring. Pick will be ignored
ERROR: Could not find peak for FtsZ ring. Pick will be ignored
ERROR: Could not find peak for FtsZ ring. Pick will be ignored
ERROR: Could not find peak for FtsZ ring. Pick will be ignored
ERROR: Could not find peak for FtsZ ring. Pick will be ignored
ERROR: Could not find peak for FtsZ ring. Pick will be ignored
ERROR: Could not find peak for FtsZ ring. Pi

Processing picks:   0%|          | 0/1 [00:00<?, ?it/s]

[array([242.15421516, 149.51515533,  52.31935079]), array([137.92313679, 267.83325955,  24.27384646])]
Calculation finished.
Total: 91 rings analysed, 68 excluded.
Data saved to CSV file in locs folder.
